# Text-Summarization with HuggingFace Transformers: Fine-Tuning & FastAPI App Deployment

In [4]:
!pip install transformers

In [5]:
## Importing Libraries
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer,TrainingArguments

In [9]:
## Load Dataset
train_data = pd.read_csv("/content/samsum-train.csv")
validation_data=pd.read_csv("/content/samsum-validation.csv")

In [10]:
## Display a sample
train_data.head(3)

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...


In [11]:
train_data.shape

(14732, 3)

In [12]:
train_data = train_data.sample(n=6000, random_state=42).reset_index(drop=True)
validation_data = validation_data.sample(n=500, random_state=42).reset_index(drop=True)

In [13]:
validation_data.shape

(500, 3)

# Data Preprocessing

In [14]:
train_data['dialogue'][0]

"Violet: hi! i came across this Austin's article and i thought that you might find it interesting\r\nViolet: <file_other>\r\nClaire: Hi! :) Thanks, but I've already read it. :)\r\nClaire: But thanks for thinking about me :)"

In [15]:
# Clean the text by removing unwanted characters
import re
def clean_text(text):
    text= re.sub(r'\r\n',' ',text) #remove carriage and new line spaces
    text= re.sub(r'\s+',' ',text) # remove extra_space
    text = re.sub(r'<.*?>',' ',text) # remove html tags <h1><p
    text = text.strip().lower() # I am loving this loving scene -->strip and convert into lower case

    return text

# Apply cleaning to dialogue and summary columns
train_data['dialogue'] = train_data['dialogue'].apply(clean_text)
train_data['summary'] = train_data['summary'].apply(clean_text)

validation_data['dialogue'] = validation_data['dialogue'].apply(clean_text)
validation_data['summary'] = validation_data['summary'].apply(clean_text)

# Display a sample after cleaning
train_data

,id,dialogue,summary
0,13811908,violet: hi! i came across this austin's articl...,violet sent claire austin's article.
1,13716431,pat: so does anyone know when the stream is go...,pat and lou are waiting for the stream but kev...
2,13810214,jane: jane: whaddya think? shona: this ur ti...,jane is updating her tinder profile tonight an...
3,13729823,"adam: do u have a map of paris? tom: yes, why?...",tom has a map of paris.
4,13681400,"frank: hi, how's the family? mike: great! sam'...","mike is happy, because sam's moved out. mike a..."
...,...,...,...
5995,13716770,"sam: so, who's ur favourite superhero? lonnie:...","lonnie's favourite superhero is batman, randy'..."
5996,13715831,geri: where are you? i'm already at the theate...,"geri is already at the theater, amy will be th..."
5997,13816234,joe: do you feel like going to the cinema? kat...,joe and kathy will meet at the cinema at 6 pm ...
5998,13828416,andy: a photo lennie took this morning from ou...,andy took a bus to work and was nearly two hou...


# Tokenization

In [16]:
# Load the tokenizer
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [17]:
def preprocess_function(examples):
    inputs = tokenizer(examples['dialogue'],padding='max_length',truncation= True,max_length=512)
    targets = tokenizer(examples['summary'],padding='summary',truncation= True,max_length=150)

In [18]:
# Preprocessing function for tokenization
def preprocess_function(examples):
    # Tokenize the dialogue and summary
    inputs = tokenizer(examples["dialogue"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(examples["summary"], padding="max_length", truncation=True, max_length=150)
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply the preprocessing
train_dataset = train_data.apply(preprocess_function, axis=1)
val_dataset = validation_data.apply(preprocess_function, axis=1)

In [19]:
train_dataset[0]

{'input_ids': [25208, 10, 7102, 55, 3, 23, 764, 640, 48, 403, 17, 77, 31, 7, 1108, 11, 3, 23, 816, 24, 25, 429, 253, 34, 1477, 25208, 10, 3, 7997, 15, 10, 7102, 55, 3, 10, 61, 2049, 6, 68, 3, 23, 31, 162, 641, 608, 34, 5, 3, 10, 61, 3, 7997, 15, 10, 68, 2049, 21, 1631, 81, 140, 3, 10, 61, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Fine Tuning Model

In [20]:
model = T5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [21]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory for checkpoints
    num_train_epochs=6,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
    evaluation_strategy="epoch",     # Ensure evaluation happens every `epoch`
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aditeemohapatra (aditeemohapatra-github) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.407500,0.365558
2,0.385100,0.350829
3,0.367900,0.347259
4,0.363200,0.342760
5,0.368100,0.341600
6,0.338700,0.341665


TrainOutput(global_step=4500, training_loss=0.7298530044555664, metrics={'train_runtime': 1812.2077, 'train_samples_per_second': 19.865, 'train_steps_per_second': 2.483, 'total_flos': 4872304852992000.0, 'train_loss': 0.7298530044555664, 'epoch': 6.0})

# Save and load Model

In [22]:
# Save the fine-tuned model
model.save_pretrained("./saved_summary_model")
tokenizer.save_pretrained("./saved_summary_model")

('./saved_summary_model/tokenizer_config.json',
 './saved_summary_model/special_tokens_map.json',
 './saved_summary_model/spiece.model',
 './saved_summary_model/added_tokens.json')

In [23]:
# Load the saved model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./saved_summary_model")
tokenizer = T5Tokenizer.from_pretrained("./saved_summary_model")

# Summarization System

In [24]:
# Ensure the model is on the correct device (GPU if available)
device = model.device  # Get the device the model is on

def summarize_dialogue(dialogue):
    dialogue = clean_text(dialogue)  # Assuming clean_text is defined
    inputs = tokenizer(dialogue, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

    # Move input tensors to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate summary
    outputs = model.generate(
        inputs["input_ids"],
        max_length=150,
        num_beams=4,
        early_stopping=True
    )

    # Decode the generated summary
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [25]:
# Test with a sample input
sample_dialogue = """
Violet: Hey Claire! I was reading an article about Austin and thought you might find it interesting!
Violet: It's about the current trends in urban development and how cities are planning for the future.
Violet: Here, let me share the link: <file_other>
Claire: Oh wow, that sounds like an insightful read. But I've actually already read that one last week.
Claire: It was really interesting though, especially the part about sustainable architecture in cities.
Claire: You know, I've been following these urban planning discussions for a while now.
Violet: Oh, I didn’t know that! Well, I’ll look for something else then, maybe something about eco-friendly cities or tech innovations.
Claire: That would be awesome! Let me know if you find something cool.
Violet: Sure, I’ll keep you posted. Thanks for the feedback!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: violet was reading an article about austin and thought it might be interesting. it's about current trends in urban development and how cities are planning for the future.


In [26]:
# Test with a dialogue on a different topic
sample_dialogue = """
John: Hey Sarah, have you seen the latest tech gadget reviews? I found this new smartwatch that's supposed to have amazing health tracking features.
John: It tracks heart rate, blood oxygen levels, sleep patterns, and even stress levels! It sounds like something right up your alley.
Sarah: That sounds really interesting! But I’ve been trying to cut down on tech distractions. I’ve heard these devices can be really overwhelming sometimes.
Sarah: I do think it’s cool that they can track so many health metrics though. I’m curious how accurate they really are.
John: Yeah, me too! There are also some new smartphones coming out with even better cameras and longer battery life. The new flagship model from XYZ brand has some insane specs.
Sarah: Ooh, I haven’t kept up with phones recently, but I’ve heard the camera quality is getting ridiculously good. It’s almost like a professional camera in your pocket now!
Sarah: Still, I feel like I’m fine with my current phone for now. I don’t really feel the need to upgrade unless something really groundbreaking comes out.
John: Totally understand that. It’s the same with me. But I think the battery life improvements are enough to make me consider it. I hate running out of battery when I’m out and about.
Sarah: That’s fair! I’m always worried about battery life too. Honestly, I think phones should last at least two full days on a single charge by now.
John: I agree! It’s so annoying when your phone dies in the middle of the day. I wonder if we’ll ever get to a point where we don’t have to charge our phones every day.
Sarah: That would be amazing! I think as tech improves, battery tech might also catch up. Let’s hope the next generation of phones can last longer!
"""

summary = summarize_dialogue(sample_dialogue)
print("Summary:", summary)

Summary: john found a new smartwatch that tracks heart rate, blood oxygen levels, sleep patterns, and stress levels. sarah hasn't kept up with phones lately. john thinks the battery life improvements are enough to make her consider it.


# Download Mode To Your Machine

In [27]:
import shutil

# Path to the directory containing the fine-tuned model
model_dir = "./saved_summary_model"

# Output zip file path
output_zip_path = "saved_summary_model.zip"

# Create a zip archive
shutil.make_archive(base_name="saved_summary_model", format="zip", root_dir=model_dir)

'/content/saved_summary_model.zip'

In [28]:
from IPython.display import FileLink

# Display a download link
FileLink(output_zip_path)

/content/saved_summary_model.zip